In [ ]:
#######################################
# Auto-reload development imports
#######################################

%load_ext autoreload
%autoreload 2

In [ ]:
#######################################
#imports
#######################################

import os, sys
from mountainlab_pytools import mlproc as mlp
from jp_ephys_viz import ephys_viz_v1

In [ ]:
dirname=os.getcwd()+'/download/NETO/2014_11_25_Pair_3_0'
bin_fname='amplifier2014-11-25T23_00_08.bin'
output_dir=os.getcwd()+'/datasets/neto_32ch_1'

In [ ]:
#######################################
# Create output directory
#######################################

datasets_dir=os.getcwd()+'/datasets'
if not os.path.exists(output_dir):
    print('Creating directory: {}'.format(output_dir))
    os.mkdir(output_dir)
else:
    print('Directory already exists: {}'.format(output_dir))

In [ ]:
# ephys.convert_array
# Convert a multi-dimensional array between various formats ('.mda', '.npy', '.dat') based on the file extensions of the input/output files
# 
# INPUTS
#   input -- Path of input array file (can be repeated for concatenation).
# 
# OUTPUTS
#   output -- Path of the output array file.
# 
# PARAMETERS
#   format -- (optional) The format for the input array (mda, npy, dat), or determined from the file extension if empty
#   format_out -- (optional) The format for the output input array (mda, npy, dat), or determined from the file extension if empty
#   dimensions -- (optional) Comma-separated list of dimensions (shape). If empty, it is auto-determined, if possible, by the input array. If second dim is -1 then it will be extrapolated from file size / first dim.
#   dtype -- (optional) The data format for the input array. Choices: int8, int16, int32, uint16, uint32, float32, float64 (possibly float16 in the future).
#   dtype_out -- (optional) The data format for the output array. If empty, the dtype for the input array is used.
#   channels -- (optional) Comma-seperated list of channels to keep in output. Zero-based indexing. Only works for .dat to .mda conversions.

def convert_array(input_fname,output_fname,*,num_channels,dtype):
    mlp.addProcess(
        'ephys.convert_array',
        dict(
            input=input_fname
        ),
        dict(
            output=output_fname
        ),
        dict(
            dimensions='{},-1'.format(num_channels),
            dtype=dtype
        ),
        dict(
        )
    )

In [ ]:
def bandpass_filter(input_fname,output_fname,*,freq_min,freq_max,samplerate):
    mlp.addProcess(
        'ephys.bandpass_filter',
        dict(
            timeseries=input_fname
        ),
        dict(
            timeseries_out=output_fname
        ),
        dict(
            freq_min=freq_min,
            freq_max=freq_max,
            samplerate=samplerate
        ),
        dict(
        )
    )

In [ ]:
P=mlp.initPipeline()

In [ ]:
with P:
    raw_fname=output_dir+'/raw0.mda'
    convert_array(dirname+'/'+bin_fname,raw_fname,num_channels=32,dtype='uint16')
    bandpass_filter(raw_fname,output_dir+'/raw.mda',freq_min=300,freq_max=6000,samplerate=30000)
    import json
    params=dict(
        samplerate=30000,
        spike_sign=-1
    )
    with open(output_dir+'/params.json','w') as f:
        json.dump(params,f)

In [ ]:
def view_dataset(dsdir,external_link=False,height=450,dataset_id='',firings=''):
    params={'view':'dataset','dataset':dsdir}
    if firings:
        params['firings']=mlp.kbucketPath(firings)
    ephys_viz_v1(params=params,title='Dataset: {}'.format(dataset_id),external_link=external_link,height=height)

In [ ]:
dsdir='kbucket://d97debc4bea2/spikeforest/datasets/neto_32ch_1'
view_dataset(dsdir,firings=dsdir+'/firings_true.mda',external_link=False)